데이터 다운로드 주소 : 
https://www.kaggle.com/pavanraj159/predicting-a-pulsar-star

![flow chart](./flow_chart.PNG)

In [ ]:
# 위의 흐름도에서 검정색 블록은 이전 실습(abalone)에서 구축해 놓은 함수를 재사용
%run ../../Week_2/07.15-abalone/AI_abalone.ipynb 

## A.1 실험용 메인 함수 정의

In [2]:
# 에폭 : 총 학습 반복 횟수 / 미니배치 크기 : 데이터 분할 단위 / 보고 주기 : 얼마나 자주 보고할 것인지
def pulsar_exec(epoch_count = 10,mb_size = 10, report=1 ): 
    # 3가지 함수를 실행
    load_pulsar_dataset()     
    init_model()            
    train_and_test(epoch_count, mb_size, report)

## B.1 데이터 적재 함수 정의

In [3]:
def load_pulsar_dataset(): 
    with open('pulsar_stars.csv') as csvfile: # with 명령어로 데이터를 열어주고 별칭 지정
        csvreader = csv.reader(csvfile) # csv.reader()로 읽어들인다.
        next(csvreader, None) # 첫 번째 행에 있는 컬럼명들을 건너뛰어 준다.
        rows = [] # 데이터를 읽어들이기 위한 빈 리스트 생성
        for row in csvreader: # 반복문을 이용한 데이터 읽어들이기
            rows.append(row) # append를 이용ㄹ한 데이터 누적 저장
    
    global data, input_count, output_count  # 전역변수 생성
    input_count, output_count = 8,1 # 입력 크기(독립 변수 8개)와 출력 크기(종속 변수 1개)
    # ns.asarray()를 활용하여 rows를 리스트 구조에서 배열로 변환하는 과정
    # -> 파이썬의 리스트 구조는 numpy에서 사용하는 다양한 일괄 산술 연산에는 비효율적 및 부적합하므로 배열로 변환
    data = np.asarray(rows, dtype = 'float32') 

#### * B.2,3 / C.1~5 / D.1 은 단층퍼셉트론에서 정의한 것을 그대로 활용 

## D.2 / E.2 후처리 과정에 대한 순전파와 역전파 함수의 재정의

In [4]:
# 시그모이드 교차 엔트로피 값을 구하는 2단계 연산 수행
def forward_postproc(output, y): 
    entropy = sigmoid_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy) # 각 수행값들에 대한 평균값 연산
    return loss, [y, output, entropy] # 바로 다음 구현할 역전파에서 사용할 리스트 [y, output, entropy]


# 순전파의 역순에 대항하는 일련의 과정을 거쳐 G_output 도출
def backprop_postproc(G_loss, aux): 
    y, output, entropy = aux # aux : 순전파 함수에서 받아온 [y, output, entropy]
    
    g_loss_entropy = 1.0 / np.prod(entropy.shape) # loss와 entropy 사이의 부분 기울기 / np.prod() : 배열의 요소들을 곱하는 함수
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y,output) # entropy와 output 사이의 부분 기울기 -> 시그모이드 교차 엔트로피의 편미분
    
    # 연쇄적 계산을 통해 손실 기울기 도출
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy

    return G_output

## D.4 정확도 계산 정의 함수

In [5]:
# 이진 판단 문제에서 정확도 : 신경망이 추정한 로짓값에 따른 판단 <-> 정답으로 주어진 판단이 일치하는 비율
def eval_accuracy(output,y):
    # output에 담긴 로짓값들의 부호를 확인하면 신경망의 판단을 확인 가능
    estimate = np.greater(output, 0) 
    # 정답으로 주어진 판단은 참일 때 1, 거짓일 때 0의 값을 갖도록 (안전하게 0.5를 기준으로 참, 거짓 구분) 
    answer = np.greater(y,0.5) # np.greater(x1, x2) : x2 기준보다 값이 크면 True 반환
    # 두 판단의 일치 여부를 비교하여 정답을 correc에 저장
    correct = np.equal(estimate, answer) # np.equal(x1, x2) : x1과 x2가 같으면 True 반환
    
    return np.mean(correct) # 평균 연산을 수행하여 옳바른 판단의 비울 측정

## E. ~ 시그모이드 관련 함수 정의

In [6]:
def relu(x): # relu()는 음수인 원소들을 모두 찾아내 0으로 대치하는 효과 -> 입력값이 음수인 경우 0, 양수는 양수를 그대로 출력
    return np.maximum(x,0) 


# 시그모이드 함수 (변형식 사용) 
def sigmoid(x): 
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))


# 시그모이드 함수의 도함수 
def sigmoid_derv(x,y):
    return y * (1-y)


# 시그모이드 교차 엔트로피 함수 (변형식 사용) 
def sigmoid_cross_entropy_with_logits(z,x): 
    return relu(x) - x * z + np.log( 1 + np.exp(-np.abs(x)))


# 시그모이드 교차 엔트로피 함수의 도함수
def sigmoid_cross_entropy_with_logits_derv(z,x):
    return -z + sigmoid(x)

## 천체의 펄서여부 판정신경망 학습시키기

In [7]:
pulsar_exec() # 이 결과에는 눈속임이 들어 있다. (불균형한 데이터셋으로 인한 착시)

Epoch 1: loss=0.154, accuracy=0.959/0.972

Final Test: final accuracy = 0.972
Epoch 2: loss=0.131, accuracy=0.966/0.972

Final Test: final accuracy = 0.972
Epoch 3: loss=0.136, accuracy=0.967/0.970

Final Test: final accuracy = 0.970
Epoch 4: loss=0.133, accuracy=0.968/0.970

Final Test: final accuracy = 0.970
Epoch 5: loss=0.121, accuracy=0.968/0.969

Final Test: final accuracy = 0.969
Epoch 6: loss=0.145, accuracy=0.968/0.974

Final Test: final accuracy = 0.974
Epoch 7: loss=0.122, accuracy=0.970/0.975

Final Test: final accuracy = 0.975
Epoch 8: loss=0.127, accuracy=0.970/0.976

Final Test: final accuracy = 0.976
Epoch 9: loss=0.125, accuracy=0.970/0.976

Final Test: final accuracy = 0.976
Epoch 10: loss=0.134, accuracy=0.968/0.976

Final Test: final accuracy = 0.976


## 새로운 예제로의 실험

In [8]:
x = np.array([130,52,0.4,0.6,3.1,20,8,72]) # 가상의 새로운 데이터 입력
output = forward_neuralnet(x) # 신경망 연산을 활용한 결과 출력
print(output) 

x_sig= sigmoid(output[0]) # 앞서 구축한 sigmoid() 통과
new_data = np.greater(x_sig,0.5) # np.greater()를 활용한 기준
print(new_data) # 결과 출력 -> False (우리가 찾는 펄서가 아니다.)

(array([-6.51508291]), array([130. ,  52. ,   0.4,   0.6,   3.1,  20. ,   8. ,  72. ]))
[False]


### ----학습데이터 증폭 및 정확도계산 정의 함수 재정의----

## A.1 메인 실행함수 재정의

In [9]:
# adjust_ratio = False : 펄서 데이터보다 별 데이터가 훨씬 많은 데이터셋을 활용 (기본값) 
# adjust_ratio = True : 펄서 데이터와 별 데이터의 비율이 같은 데이터셋을 활용
def pulsar_exec(epoch_count=10,mb_size=10, report=1, adjust_ratio = False): 
    load_pulsar_dataset(adjust_ratio)
    init_model()
    train_and_test(epoch_count, mb_size, report)

## B.1 데이터 적재 함수 재정의 

In [10]:
def load_pulsar_dataset(adjust_ratio): 
    pulsars, stars = [], [] # 펄서 데이터와 별 데이터를 담아주기 위한 빈 리스트 생성
    with open('pulsar_stars.csv') as csvfile: 
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = [] 
        for row in csvreader:
            # 만약 받아들이는 데이터의 9번째 열(종속 변수)아 '1'(픽서)인 경우 append()를 통해 pulsar에 데이터 저장
            if row[8] == '1' : pulsars.append(row) 
            # 아닌 경우(별) append()를 통해 stars 데이터 저장
            else : stars.append(row) 
    global data, input_count, output_count  
    input_count, output_count, = 8,1
    
    star_count, pulsar_count = len(stars), len(pulsars) # 전역변수 값을 각 stars와 pulsars의 행 크기에 맞춰서 지정
    
    # 매개변수 adjust의 값이 True(데이터를 늘려주는 방법을 선택)일 때, star의 비율 만큼 pulsar의 갯수를 반복적으로 복사하여 늘려준다.
    if adjust_ratio: 
        # 데이터 행의 크기가 두 배가 되는 data라는 임시 공간인 버퍼를 생성
        # 버퍼의 용도는 펄서 데이터와 별 데이터를 각각 5:5로 채워 균형을 맞춰주기 위함.
        data = np.zeros([2*star_count , 9]) 
        # 다음으로 버퍼 공간의 절반, 즉 별 데이터만큼의 공간에는 별 데이터를 행렬 형태로 넣어준다.
        data[0:star_count, :] = np.asarray(stars, dtype='float32') 

        for n in range(star_count): # star_count만큼 반복문 수행
            # 반복문을 통해 별 데이터 '이후' 행부터 하여 '펄서 데이터'가 반복적으로 저장
            # 나머지 연산자를 활용하여 펄서 데이터 수를 계속 반복 출력 / 나머지 버퍼 공간에 펄서 데이터가 들어가게 되면서 데이터의 균형이 맞춰짐
            data[star_count+n]=np.asarray(pulsars[n % pulsar_count], dtype='float32')
    
    # adjust_ratio의 인자값이 True가 아니라면
    else: 
        # 펄서와 별 데이터를 원 데이터의 크기에 맞춰 버퍼를 생성
        data = np.zeros([star_count+pulsar_count,9]) 
        # 데이터를 그대로 넣어주도록 코드를 작성
        data[0:star_count, :] = np.asarray(stars, dtype="float32")
        data[star_count:, :] = np.asarray(pulsars, dtype="float32")
    

## D.4 정확도 계산 정의 함수 재정의 

In [11]:
# 기존 함수에서는 신경망의 추측값과 실제값이 일치하는 비율에 맞춰 정확도 추출
# 하지만 이번에는 앞서 배웠던 '정밀도'와 '재현율' 그리고 'F1 수치'에 맞춰 정확도 추출
def eval_accuracy(output, y): # 정밀도와 재현율을 구하기 위해 '추측값(output)'과 '실제값(y)'을 나타내는 값들을 먼저 준비 
    # output이 0보다 더 크면 True 반환, output이 음수인 경우에는 False 반환 -> output 혹은 y가 '참'인지를 확인
    est_yes = np.greater(output, 0) 
    ans_yes = np.greater(y, 0.5) 
    # np.logical_not : 배열 원소가 조건을 만족하지 않는 경우에 '참'을 반환, 즉 output 혹은 y가 '거짓'인지를 확인 (<-> np.logical_and) 
    est_no = np.logical_not(est_yes)
    ans_no = np.logical_not(ans_yes) 
    
    # TP / FP / FN / TN
    tp = np.sum(np.logical_and(est_yes, ans_yes))
    fp = np.sum(np.logical_and(est_no, ans_yes))  
    fn = np.sum(np.logical_and(est_yes,ans_no))
    tn = np.sum(np.logical_and(est_no,ans_no))
    
    # safe_div : 나눗셈을 수행하던 중 형변환이나 0을 나누게 될 경우 나타나는 에러를 방지하기 위해 정의한 나눗셈 함수
    accuracy = safe_div(tp+tn,tp+tn+fn+fp)     
    
    precision = safe_div(tp, tp + fp)
    
    recall = safe_div(tp, tp + fn)

    f1 = 2 * safe_div(recall * precision, recall + precision)
    
    return [accuracy, precision, recall, f1]


# 안전한 계산을 위한 나눗셈 정의 (일반적인 나눗셈 연산자를 통해 연산을 진행할 경우 많은 문제를 보인다.)
def safe_div(p,q):
    p,q = float(p),float(q) # 모두 float 타입으로 형 변환
    
    # 만약 q, 즉 분모가 너무 작은 수라서 나눗셈을 진행할 때 에러가 날 수도 있겠다는 경우라면, 
    # 혹은 조건을 걸어 '절댓값' 자체가 너무 작은 경우라면,
    # -> np.sign (0이면 0, 양수는 1, 음스는 -1을 출력) 함수를 활용하여 분자의 값을 0, -1, 1 로 조정하여 나눗셈을 진행.
    if np.abs(q) < 1.0e-20: 
        return np.sign(p)
    
    return p / q

## B.3 출력문 수정을 위한 실행함수 재정의

In [12]:
# 이 함수는 abalone_exec()에서 정의한 함수지만, 
# run_test()에서 호출하는 eval_accuracy() 함수에서 반환값 형식이 accuracy 하나에서 4가지 평가지표로 달라졌기 때문에 재정의가 필요
def train_and_test(epoch_count, mb_size, report): # 이전과 동일
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count): # accuracy를 이미 앞에서 구축했기 때문에 따로 정확도를 저장하지 않는다.
        losses = []
        
        for n in range(step_count): # 이전과 동일
            train_x, train_y = get_train_data(mb_size, n)
            loss,_ = run_train(train_x,train_y)
            losses.append(loss)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x,test_y) 
            # 각 지표는 쉼표를 기준으로 구분하도록 하고, join 함수를 통해 형식과 어느정도 크기 정보를 전달
            acc_str = ','.join(['%5.3f']*4) % tuple(acc) 
            
            # 4가지 평가지표 리스트 형식으로 그에 맞게 문자열 포멧팅 형식을 변경
            print('Epoch{}:loss = {:5.3f},result={}'.format(epoch+1,np.mean(losses),acc_str))
            
    acc = run_test(test_x, test_y)
    acc_str = ','.join(['%5.3f']*4) % tuple(acc) # 4가지 요소 모두 동일한 규격(포멧팅) 지정
    print('\n Final Test: final result = {}'.format(acc_str))
    
    

In [13]:
#acc_str = ','.join(['%5.3f']*4) % tuple([1,2,3,4])

## 실행하기 (adjust_ratio = False)

In [14]:
pulsar_exec(adjust_ratio = False) # 불균형한 종속변수의 분포를 보인다.

Epoch1:loss = 0.136,result=0.972,0.794,0.879,0.835
Epoch2:loss = 0.124,result=0.973,0.732,0.959,0.830
Epoch3:loss = 0.127,result=0.966,0.629,0.990,0.770
Epoch4:loss = 0.129,result=0.973,0.717,0.970,0.824
Epoch5:loss = 0.126,result=0.961,0.841,0.752,0.794
Epoch6:loss = 0.134,result=0.973,0.769,0.911,0.834
Epoch7:loss = 0.130,result=0.975,0.757,0.949,0.842
Epoch8:loss = 0.132,result=0.972,0.723,0.955,0.823
Epoch9:loss = 0.120,result=0.975,0.754,0.953,0.842
Epoch10:loss = 0.123,result=0.971,0.692,0.982,0.812

 Final Test: final result = 0.971,0.692,0.982,0.812


## 실행하기 (adjust_ratio = True)

In [15]:
pulsar_exec(adjust_ratio=True) # 균형잡힌 종속변수의 분포를 보인다. -> 정확도 감소, F1 수치 증가, 정밀도와 재현율에는 약간의 편차가 존재.

Epoch1:loss = 0.402,result=0.907,0.926,0.892,0.909
Epoch2:loss = 0.371,result=0.916,0.892,0.937,0.914
Epoch3:loss = 0.351,result=0.915,0.848,0.980,0.909
Epoch4:loss = 0.374,result=0.917,0.854,0.976,0.911
Epoch5:loss = 0.374,result=0.900,0.933,0.875,0.903
Epoch6:loss = 0.354,result=0.922,0.870,0.970,0.918
Epoch7:loss = 0.371,result=0.919,0.907,0.929,0.918
Epoch8:loss = 0.344,result=0.925,0.897,0.950,0.923
Epoch9:loss = 0.363,result=0.910,0.833,0.984,0.902
Epoch10:loss = 0.359,result=0.925,0.888,0.959,0.922

 Final Test: final result = 0.925,0.888,0.959,0.922


-> 데이터 간의 균형은 매우 중요하며, 무조건적인 단순 비교 정확도 추출보다는 다양한 지표를 활용해야 한다. 